<center><div class="alert alert-block alert-info" style=" line-height: 1.7em; font-family: Verdana;">
    <b style="font-size: 18px; color:green"> &nbsp; Bengali.AI Speech Recognition : Our very first submission</b><br><br><b style="font-size: 18px; color:green">HELLO EVERYONE!</b><br>
</div></center>

In this Notebook we'll try to submit our first submission by using a publicly available model.

We'll be using this [BanglaASR](https://huggingface.co/bangla-speech-processing/BanglaASR) model. It is actually a finetuned [whisper](https://openai.com/research/whisper) model on rained [Bangla Mozilla Common Voice Dataset](https://arxiv.org/abs/2206.14053). It's reported performance is 4.58% on 7k validation set which seems pretty good to be a starting point. So let's start with that.

I'll be trying to use the other publicly available models one by one and see how they perform. Then we'll get an idea from where we can start the training/finetuning on this huge dataset.

# Imports

In [7]:
import os
import librosa
import torch
import torchaudio
import numpy as np

from transformers import WhisperTokenizer
from transformers import WhisperProcessor
from transformers import WhisperFeatureExtractor
from transformers import WhisperForConditionalGeneration
from tqdm import tqdm
import pandas as pd
import soundfile as sf
from pydub import AudioSegment

# Load the model

While submitting notebooks, you need to have internet disabled. So you can't download models from huggingface directly. One workaround is to download the models from huggingface, upload them to kaggle as datasets and then use them!

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_path = "/kaggle/input/bangla-speech-processing-banglaasr/BanglaASR"
feature_extractor = WhisperFeatureExtractor.from_pretrained(model_path)
tokenizer = WhisperTokenizer.from_pretrained(model_path)
processor = WhisperProcessor.from_pretrained(model_path)
model = WhisperForConditionalGeneration.from_pretrained(model_path).to(device)

# Demo inference

We'll write an inference function where we'll give the path as input and it will return the transcription!

First we'll need to convert the audio sampling rate to 16k since WhisperFeatureExtractor requires audios to be sampled at 16k.

In [9]:
#This code is modified from the source code provided here https://huggingface.co/bangla-speech-processing/BanglaASR
def inference_fn(path):
    speech_array, sampling_rate = sf.read(mp3_path)
    speech_array = librosa.resample(np.asarray(speech_array), orig_sr=sampling_rate, target_sr=16000)
    input_features = feature_extractor(speech_array, sampling_rate=16000, return_tensors="pt").input_features
    predicted_ids = model.generate(inputs=input_features.to(device))[0]
    transcription = processor.decode(predicted_ids, skip_special_tokens=True)
    return transcription


# Inference

In [10]:
mp3_path = "/kaggle/input/rangpur-1/rec_01_audio_0.wav"
print(f"File name :",mp3_path)
AudioSegment.from_file(mp3_path)

File name : /kaggle/input/rangpur-1/rec_01_audio_0.wav


In [11]:
import os
test_path = "/kaggle/input/rangpur-1"
files = os.listdir(test_path)
ids = []
sentences = []
for file in tqdm(files):
    ids.append(file.split(".")[0])
    mp3_path = os.path.join(test_path,file)
    prediction = inference_fn(mp3_path)
    
    #sanity check
    if len(prediction)==0:
        prediction = "<>"
    
    sentences.append(prediction)

  0%|          | 0/1000 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (448) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 1000/1000 [4:57:11<00:00, 17.83s/it] 


Predictions ready. Let's make the csv file

In [12]:
df1 = pd.DataFrame({"External_ID":ids,"Contents":sentences})
df1.head()

,External_ID,Contents
0,rec_22_audio_69,তোই কিজন্য মানুষের মটার চুলী বের বিক্তিমানুষক ...
1,rec_19_audio_24,গড়িপ মারির যে কেমনের জীবন যাপন করে শ্বক জিন স...
2,rec_05_audio_0,"হেরে বাবলু! কেমন আসিসলে! আসিত ভালো বাছে, তেন ভ..."
3,rec_08_audio_32,যে মুহুদির দেখান তুর্ব ভালোই আশ্বর্যাল সবসময় ...
4,rec_18_audio_10,"আরেখানে কাম হলেল কে, খাওদার আয়োজনকন্তু, সেইভা..."


In [13]:
df1

,External_ID,Contents
0,rec_22_audio_69,তোই কিজন্য মানুষের মটার চুলী বের বিক্তিমানুষক ...
1,rec_19_audio_24,গড়িপ মারির যে কেমনের জীবন যাপন করে শ্বক জিন স...
2,rec_05_audio_0,"হেরে বাবলু! কেমন আসিসলে! আসিত ভালো বাছে, তেন ভ..."
3,rec_08_audio_32,যে মুহুদির দেখান তুর্ব ভালোই আশ্বর্যাল সবসময় ...
4,rec_18_audio_10,"আরেখানে কাম হলেল কে, খাওদার আয়োজনকন্তু, সেইভা..."
...,...,...
995,rec_09_audio_19,নাগজিস্টকন উদ্যোখন করেন।
996,rec_04_audio_6,একটিরিশবছর দুইমাস তিনদিন।নাত কী গুরবের নাক শ্র...
997,rec_16_audio_7,"-না এলাহ হয় না! এলাহ বহে সুনু কিন্নের গলে, কা..."
998,rec_21_audio_40,"সেলে, ঘোঁট বমশাকারী জেলেন, হারা খানিককাচা শোনা..."


In [14]:
df1.to_csv("submission1.csv",index=False)

In [15]:
#=====================================================================================================================================

In [16]:
import os
test_path = "/kaggle/input/rangpur-2"
files = os.listdir(test_path)
ids = []
sentences = []
for file in tqdm(files):
    ids.append(file.split(".")[0])
    mp3_path = os.path.join(test_path,file)
    prediction = inference_fn(mp3_path)
    
    #sanity check
    if len(prediction)==0:
        prediction = "<>"
    
    sentences.append(prediction)

  0%|          | 0/322 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (448) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 322/322 [1:30:48<00:00, 16.92s/it]


In [17]:
df2 = pd.DataFrame({"External_ID":ids,"Contents":sentences})
df2.head()

,External_ID,Contents
0,rec_22_audio_120,কেম্বুলেজ দুর্ডে আমি বাড়া করেছেন।িসেবে যাশাস্...
1,rec_27_audio_3,আনী দৃশ্যের জমিদা মা। ধানেরাবাদ নে ভুটেরাবাদ। ...
2,rec_26_audio_19,যে দোকান্টা ফাঁকা থাকতে? বেচা কেন যে আগে আট্টো...
3,rec_24_audio_50,"রয়েছে যখরণ, জনীন্ত্র যখরণ- রাজিয়া- না, ছোটোক..."
4,rec_22_audio_123,মসক- বাবুল মা-আরশীদের ভাই- প্রতিশ আর হল মিটু ব...


In [18]:
df2

,External_ID,Contents
0,rec_22_audio_120,কেম্বুলেজ দুর্ডে আমি বাড়া করেছেন।িসেবে যাশাস্...
1,rec_27_audio_3,আনী দৃশ্যের জমিদা মা। ধানেরাবাদ নে ভুটেরাবাদ। ...
2,rec_26_audio_19,যে দোকান্টা ফাঁকা থাকতে? বেচা কেন যে আগে আট্টো...
3,rec_24_audio_50,"রয়েছে যখরণ, জনীন্ত্র যখরণ- রাজিয়া- না, ছোটোক..."
4,rec_22_audio_123,মসক- বাবুল মা-আরশীদের ভাই- প্রতিশ আর হল মিটু ব...
...,...,...
317,rec_22_audio_85,যখন পরিক্ষাদিবাজ্যায় তামিয়াবার পথাদি ছিলেন জ...
318,rec_24_audio_70,রাজশ সেস্কেও!
319,rec_22_audio_87,এইরকম যেকদিন পরীক্ষে দিবের কিছু ঐকজাধিন এইরকম ...
320,rec_23_audio_35,"আবাকি খাবিউ ক্রিয়ান্ম ফেডেন, ফেডেন হামারু মন্..."


In [19]:
df2.to_csv("submission2.csv",index=False)

In [20]:
#=====================================================================================================================================

In [21]:
values1 = df1[['External_ID','Contents']]
values2 = df2[['External_ID','Contents']]


dataframes = [values1, values2]

join = pd.concat(dataframes)
join

,External_ID,Contents
0,rec_22_audio_69,তোই কিজন্য মানুষের মটার চুলী বের বিক্তিমানুষক ...
1,rec_19_audio_24,গড়িপ মারির যে কেমনের জীবন যাপন করে শ্বক জিন স...
2,rec_05_audio_0,"হেরে বাবলু! কেমন আসিসলে! আসিত ভালো বাছে, তেন ভ..."
3,rec_08_audio_32,যে মুহুদির দেখান তুর্ব ভালোই আশ্বর্যাল সবসময় ...
4,rec_18_audio_10,"আরেখানে কাম হলেল কে, খাওদার আয়োজনকন্তু, সেইভা..."
...,...,...
317,rec_22_audio_85,যখন পরিক্ষাদিবাজ্যায় তামিয়াবার পথাদি ছিলেন জ...
318,rec_24_audio_70,রাজশ সেস্কেও!
319,rec_22_audio_87,এইরকম যেকদিন পরীক্ষে দিবের কিছু ঐকজাধিন এইরকম ...
320,rec_23_audio_35,"আবাকি খাবিউ ক্রিয়ান্ম ফেডেন, ফেডেন হামারু মন্..."


In [22]:
join.to_csv("3.1 rangpur-banglaWhisper-inferences.csv")

In [23]:
print('done')

done
